In [4]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn-v0_8')

In [5]:
# задаем значение random_state, которое будем использовать в моделях 
RANDOM_STATE = 42

## Описание задачи.

* Необходимо обучить две модели: логистическую регрессию и случайный лес.  
* Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации.  
* Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.
* В качестве метрики будем использовать F1-score.
* Предварительная обработка не требуется, данные уже закодированы и нормализованы.

### Загружаем данные.

In [6]:
data = pd.read_csv('data/_train_sem09__1_.zip', compression='zip')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


### По условиям предобработка не требуется.
* Данные предварительно закодированы и нормализованы.
* Проверяем отсутвие пропусков.

In [7]:
data.isna().sum().sum()

0

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


### Выделяем целевой признак. Проверяем сбалансированность.

In [9]:
X = data.drop(columns='Activity')
y = data['Activity']
X.shape, y.shape

((3751, 1776), (3751,))

In [10]:
y.value_counts(True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

> Целевой признак бинарный. Выборка достаточно сбалансирована. Стратификация не требуется.

### Для обучения и оценки моделей разбиваем датасет на трейн и тест.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

### Создаем модель логистической регрессии с параметрами по умолчанию.
* В качестве метрики будем использовать F1_score 

In [12]:
log_reg = linear_model.LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)

y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')

accuracy на тестовом наборе: 0.75
f1_score на трейн: 0.89
f1_score на тест: 0.79


### Для поиска оптимальных гиперпараметров логистичекой регрессии используем RandomizedSearchCV
* метрика для оценки F1_score

In [13]:
from sklearn.model_selection import RandomizedSearchCV

# задаем сетку гиперпараметров
param_distributions = [{'penalty': ['l2'] ,
              'solver': ['lbfgs', 'sag', 'saga', 'newton-cg', 'liblinear'],
               'C': list(np.linspace(0.01, 1, 10))
               },
                       {'penalty': ['l1'] ,
              'solver': ['liblinear', 'saga'],
              'C': list(np.linspace(0.01, 1, 10))
               },]
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=RANDOM_STATE, max_iter=10000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50,
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: user 50.1 s, sys: 504 ms, total: 50.6 s
Wall time: 6min 12s
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.45}


> * Выбранные RandomizedSearchCV гиперпараметры дают f1_score анлогичную бейзмодел (с парараметрами по умолчанию).  
> * Едва заметно улучшение accuracy 0.76 vs 0.75  
> * Mожно отметить, что подбирались гиперпараметры для минизации лосс-функции, что не всегда равносильно лучшему f1_score.

### Используем Hyperopt для подбора параметров на основе баесовской оптимизации

In [14]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, space_eval
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

In [15]:
# зададим пространство поиска гиперпараметров
space_lr = hp.choice('penalty_ch',[
                     {'penalty': 'l2',
                      'solver': hp.choice('solver1', ['lbfgs', 'sag', 'saga', 'newton-cg', 'liblinear']),
                      'C': hp.uniform('C1', 0.01, 1)},
                     {'penalty': 'l1',
                      'solver': hp.choice('solver2', ['liblinear', 'saga']),
                      'C': hp.uniform('C2', 0.01, 1)},
                     ])

In [16]:
# зададим функцию, для оценки используем f1_score
def hyperopt_lr(params, X_train=X_train, y_train=y_train, random_state=RANDOM_STATE):
    params = {'penalty': params['penalty'], 
              'solver': params['solver'],
              'C': params['C']
              }
    log_reg = linear_model.LogisticRegression(**params, random_state=random_state, max_iter = 10000)
    log_reg.fit(X_train, y_train)
    score = metrics.f1_score(y_train, log_reg.predict(X_train))
    return -score

In [17]:
%%time
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(RANDOM_STATE)
         )
print(f'Наилучшие значения гиперпараметров {space_eval(space_lr, best)}')

100%|██████████| 50/50 [06:58<00:00,  8.38s/trial, best loss: -0.8868274582560297]
Наилучшие значения гиперпараметров {'C': 0.9940195730782141, 'penalty': 'l2', 'solver': 'liblinear'}
CPU times: user 9min 5s, sys: 58.8 s, total: 10min 4s
Wall time: 6min 58s


In [18]:
# рассчитаем для тестовой выборки
log_reg_ho = linear_model.LogisticRegression(max_iter = 10000,
                                             penalty='l2',
                                             solver='liblinear',
                                             C=0.99)
log_reg_ho.fit(X_train, y_train)

y_train_pred = log_reg_ho.predict(X_train)
y_test_pred = log_reg_ho.predict(X_test)
print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')


accuracy на тестовом наборе: 0.75
f1_score на трейн: 0.89
f1_score на тест: 0.79


> * Выбранные Hyperopt гиперпараметры дают f1_score аналогичную бейзмодел (с парараметрами по умолчанию).   
> * f1_score на трейн значительно превышает тест, попробуем использовать для оценки кросс-валидацию.

In [19]:
# задаем функцию оценки с использованием кросс-валидации
def hyperopt_lr_cv(params, cv=5, X_train=X_train, y_train=y_train, random_state=RANDOM_STATE):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver'],
              'C': params['C']
              }
  
    log_reg = linear_model.LogisticRegression(**params, random_state=random_state, max_iter = 10000)
    
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(log_reg, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [20]:
%%time
trials_cv = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr_cv, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials_cv, # логирование результатов
          rstate=np.random.default_rng(RANDOM_STATE)
         )
print(f'Наилучшие значения гиперпараметров {space_eval(space_lr, best)}')

100%|██████████| 50/50 [07:57<00:00,  9.55s/trial, best loss: -0.7866466124784772]
Наилучшие значения гиперпараметров {'C': 0.05084775379720359, 'penalty': 'l2', 'solver': 'lbfgs'}
CPU times: user 586 ms, sys: 660 ms, total: 1.25 s
Wall time: 7min 57s


In [21]:
# рассчитаем для тестовой выборки
log_reg_ho_vs = linear_model.LogisticRegression(max_iter = 10000,
                                          penalty='l2',
                                          solver='lbfgs',
                                          C=0.05)
log_reg_ho_vs.fit(X_train, y_train)

y_train_pred = log_reg_ho_vs.predict(X_train)
y_test_pred = log_reg_ho_vs.predict(X_test)
print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')


accuracy на тестовом наборе: 0.76
f1_score на трейн: 0.84
f1_score на тест: 0.79


> * Нам по прежнему не удалось добиться улучшения f1_score.
> * Возможно качество последней модели чуть-чуть лучше, так как f1_score на трейн и тест ближе, и accuracy хоть и на один пункт, но выше бейзмодел.
> * В целом уже на этапе RandomizedSearchCV стало понятно, что подбор гиперпараметров в данном случае не дает существенного улчшения метрики для логистической регрессии.
> * Можно пробовать другую модель.

### Создаем Случаный лес - RandomForestClassifier с параметрами по умолчанию

In [22]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=RANDOM_STATE)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')

accuracy на тестовом наборе: 0.80
f1_score на трейн: 1.00
f1_score на тест: 0.83


> * RandomForestClassifier показывает метрики лучше, чем удалось добиться с LogisticRegression.

### Для подбора гиперпараметров используем GridSearchCV

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': list(range(50, 200, 25)),
              'min_samples_leaf': [1, 2, 5, 7],
              'max_depth': list(np.linspace(10, 30, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=RANDOM_STATE), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train) 
y_train_pred = grid_search_forest.predict(X_train)
y_test_pred = grid_search_forest.predict(X_test)


print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')
print(f"Наилучшие значения гиперпараметров: {grid_search_forest.best_params_}")

CPU times: user 1.65 s, sys: 212 ms, total: 1.86 s
Wall time: 1min 6s
accuracy на тестовом наборе: 0.80
f1_score на трейн: 0.99
f1_score на тест: 0.83
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 75}


> * Найденные GridSearchCV гиперпараметры не показывают существенного отличия метрики от базовой.

### Используем библиотеку Optuna для поиска оптимальных гиперпараметров

In [24]:
import optuna
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.2.0


In [44]:
# задаем оценивающую фукцию с сеткой гиперпараметров для алгоритма случайного леса 
def optuna_rf_cv(trial, X_train=X_train, y_train=y_train, cv=5, random_state=RANDOM_STATE):
    # определяем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 50, 200, 2)
    max_depth = trial.suggest_int('max_depth', 10, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10, 1)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss'])

    # создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state,
                                          criterion=criterion
                                          )
    # используем крос-валидацию
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1).mean()

    return score
  

In [56]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
stud_cv = optuna.create_study(study_name="RandomForestClassifier_cv", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
stud_cv.optimize(optuna_rf_cv, n_trials=50)

[I 2023-06-01 15:44:52,997] A new study created in memory with name: RandomForestClassifier_cv
[I 2023-06-01 15:44:54,352] Trial 0 finished with value: 0.7914935481406152 and parameters: {'n_estimators': 72, 'max_depth': 18, 'min_samples_leaf': 10, 'criterion': 'log_loss'}. Best is trial 0 with value: 0.7914935481406152.
[I 2023-06-01 15:44:55,202] Trial 1 finished with value: 0.8013565082088345 and parameters: {'n_estimators': 60, 'max_depth': 10, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8013565082088345.
[I 2023-06-01 15:44:56,019] Trial 2 finished with value: 0.7898349947095099 and parameters: {'n_estimators': 60, 'max_depth': 18, 'min_samples_leaf': 10, 'criterion': 'entropy'}. Best is trial 1 with value: 0.8013565082088345.
[I 2023-06-01 15:44:56,866] Trial 3 finished with value: 0.803002161741278 and parameters: {'n_estimators': 52, 'max_depth': 23, 'min_samples_leaf': 2, 'criterion': 'gini'}. Best is trial 3 with value: 0.803002161741278.
[I 

CPU times: user 831 ms, sys: 627 ms, total: 1.46 s
Wall time: 1min 4s


In [57]:
# выводим результаты на обучающей выборке
print(f"Наилучшие значения гиперпараметров: {stud_cv.best_params}")
print(f"f1_score на обучающем наборе: {stud_cv.best_value:.2f}")

Наилучшие значения гиперпараметров: {'n_estimators': 198, 'max_depth': 21, 'min_samples_leaf': 2, 'criterion': 'entropy'}
f1_score на обучающем наборе: 0.82


In [58]:
# рассчитаем точность для тестовой выборки
model_b = ensemble.RandomForestClassifier(**stud_cv.best_params, random_state=RANDOM_STATE)
model_b.fit(X_train, y_train)

y_train_pred = model_b.predict(X_train)
y_test_pred = model_b.predict(X_test)

print(f'accuracy на тестовом наборе: {metrics.accuracy_score(y_test, y_test_pred):.2f}')
print(f'f1_score на трейн: {metrics.f1_score(y_train, y_train_pred):.2f}')
print(f'f1_score на тест: {metrics.f1_score(y_test, y_test_pred):.2f}')

accuracy на тестовом наборе: 0.82
f1_score на трейн: 1.00
f1_score на тест: 0.85


> * Гиперпараметры найденные Optuna дают некоторое улучшение, целевая метрика f1_score повысилась до 0.85.
> * Аccuracy также показывает небольшее улучшение до 0.82.
> * Optuna позволяет оценить более широкую сетку гиперпараметров за меньшее время.

### Итоговый вывод.
__Из рассмотренных способов подбора параметров Optuna дает возможность быстрого поиска оптимальных значений по широкой сетке параметров.__  

__В данной работе методы подбора гиперпараметров смогли принести минимальные улучшения моделей по сравнению с базовыми (заданными по умолчанию гиперпараметрами).__ 

__Это может быть связано с тем, что гиперпараметры по умолчанию использованных моделей подобраны отимальными для большого круга задач.__
